In [ ]:
import keras
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.models import Sequential
from keras.preprocessing.image import img_to_array, load_img
from keras.callbacks import ModelCheckpoint
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
keras.__version__

In [ ]:
# Get images
image = img_to_array(load_img('golf.jpg'))
image = np.array(image, dtype=float)

In [ ]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

In [ ]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [ ]:
# Finish model
filepath="model_v1.h5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1,save_best_only=True,mode='max')
model.compile(optimizer='rmsprop', loss='mse',metrics=['acc'])

In [ ]:
history = model.fit(x=X,y=Y,batch_size=1,epochs=1000,callbacks=[checkpoint], verbose=2)

In [ ]:
acc = history.history['acc']
loss = history.history['loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.title('Training accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))